<a href="https://colab.research.google.com/github/shaharec/NLP_BraveTogether/blob/main/NewModel/Text_classification_with_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone "https://github.com/ilanigi/Brave_Together_Scraper.git"

Cloning into 'Brave_Together_Scraper'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 137 (delta 59), reused 55 (delta 20), pack-reused 0
Receiving objects: 100% (137/137), 2.27 MiB | 7.29 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [2]:
# import libraries
import pandas as pd


In [9]:
df_labeld_data = pd.read_csv("/content/Brave_Together_Scraper/labeled_data.csv");
df.info

<bound method DataFrame.info of        Unnamed: 0  ...                                              tweet
0               0  ...  !!! RT @mayasolovely: As a woman you shouldn't...
1               1  ...  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2               2  ...  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3               3  ...  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4               4  ...  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...           ...  ...                                                ...
24778       25291  ...  you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779       25292  ...  you've gone and broke the wrong heart baby, an...
24780       25294  ...  young buck wanna eat!!.. dat nigguh like I ain...
24781       25295  ...              youu got wild bitches tellin you lies
24782       25296  ...  ~~Ruffled | Ntac Eileen Dahlia - Beautiful col...

[24783 rows x 7 columns]>

#Data frame 
1. class - 1 for holocaust denial
2. 

In [7]:
df.head(10)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5,5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just..."
6,6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up ..."
7,7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...
8,8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; ..."
9,9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Maria..."


In [14]:
df["tweet"][500],df["class"][500]

('"I\'m probably your main bitch chocolate dipped cinnamon apple"', 1)

In [1]:
!git status

fatal: not a git repository (or any of the parent directories): .git
